# Use BERT Representations with LogisticRegression Softmax Classifier

In [1]:
from collections import Counter
import os
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from transformers import BertTokenizer, BertModel, BertForSequenceClassification


import dataset
import vsm
import sst

In [2]:
TWITTER = 2
TWITTER_AIRLINES = 3
TWITTER_APPLE = 4

In [3]:
twitter_train, twitter_validate, twitter_test =  dataset.dataset_reader(TWITTER)
[twitter_train, twitter_validate, twitter_test] = list(map(lambda ds : dataset.prune_columns(2, ds), [twitter_train, twitter_validate, twitter_test]))

In [4]:
airline_train, airline_validate, airline_test =  dataset.dataset_reader(TWITTER_AIRLINES)
[airline_train, airline_validate, airline_test] = list(map(lambda ds : dataset.prune_columns(3, ds), [airline_train, airline_validate, airline_test]))

In [5]:
apple_train, apple_validate, apple_test =  dataset.dataset_reader(TWITTER_APPLE)
[apple_train, apple_validate, apple_test] = list(map(lambda ds : dataset.prune_columns(4, ds), [apple_train, apple_validate, apple_test]))

In [6]:
# Rename labels for consistency
def rename_sentiment(sentiment):
        if sentiment in ["5", "positive"]:
            return "Positive"
        elif sentiment in ["3", "neutral"]:
            return "Neutral"
        elif sentiment in ["1", "negative"]:
            return "Negative"
        elif sentiment in ["not_relevant"]:
            return "Irrelevant"
        else:
            return sentiment


In [7]:
airline_train['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in airline_train['sentiment']]).set_index(airline_train.index)
airline_validate['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in airline_validate['sentiment']]).set_index(airline_validate.index)
airline_test['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in airline_test['sentiment']]).set_index(airline_test.index)

In [8]:
apple_train['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in apple_train['sentiment']]).set_index(apple_train.index)
apple_validate['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in apple_validate['sentiment']]).set_index(apple_validate.index)
apple_test['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in apple_test['sentiment']]).set_index(apple_test.index)

In [9]:
# cat the datasets
train_ds = pd.concat([twitter_train,airline_train,apple_train],axis=0)
validate_ds = pd.concat([twitter_validate,airline_validate,apple_validate],axis=0)
test_ds = pd.concat([twitter_test,airline_test,apple_test],axis=0)

In [10]:
bert_weights_name = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_weights_name)
bert_model = BertModel.from_pretrained(bert_weights_name)
# model = BertForSequenceClassification.from_pretrained(bert_weights_name)
# Unique values of sentiment
twitter_sentiment_labels = train_ds['sentiment'].unique()

In [11]:
train_ds.size, validate_ds.size, test_ds.size

(402588, 17112, 145548)

In [12]:
%%time
bert_experiment_full = sst.experiment(
    [train_ds], # 
    dataset.hf_cls_phi,
    dataset.fit_softmax_classifier,
    assess_dataframes=[validate_ds],
    vectorize=False)

              precision    recall  f1-score   support

  Irrelevant      0.529     0.199     0.289       181
    Negative      0.728     0.871     0.793      1297
     Neutral      0.641     0.531     0.581       813
    Positive      0.639     0.635     0.637       561

    accuracy                          0.685      2852
   macro avg      0.634     0.559     0.575      2852
weighted avg      0.673     0.685     0.670      2852

CPU times: user 7h 6min 22s, sys: 2min 58s, total: 7h 9min 21s
Wall time: 1h 14min 1s


In [13]:
bert_experiment_full.keys()

dict_keys(['model', 'phi', 'train_dataset', 'assess_datasets', 'predictions', 'metric', 'scores'])

In [14]:
bert_experiment_full['scores']

[0.5750065613628772]

In [15]:
bert_experiment_full['metric']

'safe_macro_f1'

In [16]:
bert_experiment_full['model']

LogisticRegression(multi_class='ovr', solver='liblinear')

# Test BERT trained on Tweets on test set

In [17]:
def predict_one_bert(text):
    # List of tokenized examples:
    X = [bert_experiment_full['phi'](text)]
    # Standard `predict` step on a list of lists of str:
    preds = bert_experiment_full['model'].predict(X)
    # Be sure to return the only member of the predictions,
    # rather than the singleton list:
    return preds[0]

In [18]:
# %% time
# twitter_test['prediction'] = twitter_test['text'].apply(predict_one_bert)

In [19]:
# import importlib
# importlib.reload(sst)

In [20]:
%%time
bert_test = sst.evaluate(
    bert_experiment_full['model'],
    bert_experiment_full['phi'],
    assess_dataframes=[test_ds],
    vectorizer=bert_experiment_full['assess_datasets'][0]['vectorizer'],
    vectorize=False
)

              precision    recall  f1-score   support

  Irrelevant      0.479     0.233     0.313      3911
    Negative      0.607     0.742     0.668      7851
     Neutral      0.549     0.524     0.536      6008
    Positive      0.581     0.629     0.604      6488

    accuracy                          0.576     24258
   macro avg      0.554     0.532     0.530     24258
weighted avg      0.565     0.576     0.561     24258

CPU times: user 2h 24min 10s, sys: 56.7 s, total: 2h 25min 6s
Wall time: 24min 11s


In [21]:
type(bert_test['predictions'][0])

numpy.ndarray

In [22]:
predictions_fname ='results/BERT_predictions_on_combined_twitter_test.csv'
df = bert_test['predictions'][0]
pd.DataFrame(df).to_csv(predictions_fname)

In [23]:
encoding_fname ='results/BERT_encodings_on_combined_twitter_test.csv'
encoded_test = bert_test['assess_datasets'][0]
pd.DataFrame(df).to_csv(encoding_fname)

In [24]:
predictions_df = pd.DataFrame(df)
predictions_df = predictions_df.set_index(test_ds.index)
predictions_df

,0
1,Negative
2,Negative
11,Positive
13,Neutral
17,Neutral
...,...
3826,Negative
3835,Positive
3845,Neutral
3876,Neutral


In [25]:
test_ds['BERT_sentiment'] = predictions_df

In [26]:
test_ds

,dataset,tweet_id,text,sentiment,entity,airline,BERT_sentiment
1,twitter_sentiment,1,I am coming to the borders and I will kill you...,Positive,2401.0,NaN,Negative
2,twitter_sentiment,2,im getting on borderlands and i will kill you ...,Positive,2401.0,NaN,Negative
11,twitter_sentiment,3,was,Positive,2402.0,NaN,Positive
13,twitter_sentiment,4,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...",Neutral,2403.0,NaN,Neutral
17,twitter_sentiment,5,"I-Hard like me, RARE LONDON DE, HANDSOME 2011,...",Neutral,2403.0,NaN,Neutral
...,...,...,...,...,...,...,...
3826,twitter_apple,623499356,RT @iLoveMyMom98: I've tried turning it off an...,Negative,NaN,NaN,Negative
3835,twitter_apple,623499365,@afrobugeisha @Apple watch it!!!!!,Neutral,NaN,NaN,Positive
3845,twitter_apple,623499375,media reports say that @Apple is hiring pros f...,Neutral,NaN,NaN,Neutral
3876,twitter_apple,623499406,Apple Is Warming Up To Social Media: Apple is ...,Neutral,NaN,NaN,Neutral


In [27]:
test_predictions_fname ='results/BERT_predictions_added_to_combined_twitter_test.csv'
test_ds.to_csv(test_predictions_fname)

In [28]:
correct = test_ds[test_ds['sentiment'] == test_ds['BERT_sentiment']]

In [29]:
correct

,dataset,tweet_id,text,sentiment,entity,airline,BERT_sentiment
11,twitter_sentiment,3,was,Positive,2402.0,NaN,Positive
13,twitter_sentiment,4,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...",Neutral,2403.0,NaN,Neutral
17,twitter_sentiment,5,"I-Hard like me, RARE LONDON DE, HANDSOME 2011,...",Neutral,2403.0,NaN,Neutral
19,twitter_sentiment,6,this was the first Borderlands session in a lo...,Positive,2404.0,NaN,Positive
22,twitter_sentiment,7,that I was the first real borderlands session ...,Positive,2404.0,NaN,Positive
...,...,...,...,...,...,...,...
3823,twitter_apple,623499353,13% left what even is this shit @apple,Negative,NaN,NaN,Negative
3826,twitter_apple,623499356,RT @iLoveMyMom98: I've tried turning it off an...,Negative,NaN,NaN,Negative
3845,twitter_apple,623499375,media reports say that @Apple is hiring pros f...,Neutral,NaN,NaN,Neutral
3876,twitter_apple,623499406,Apple Is Warming Up To Social Media: Apple is ...,Neutral,NaN,NaN,Neutral


In [30]:
incorrect = test_ds[test_ds['sentiment'] != test_ds['BERT_sentiment']]

In [31]:
incorrect

,dataset,tweet_id,text,sentiment,entity,airline,BERT_sentiment
1,twitter_sentiment,1,I am coming to the borders and I will kill you...,Positive,2401.0,NaN,Negative
2,twitter_sentiment,2,im getting on borderlands and i will kill you ...,Positive,2401.0,NaN,Negative
37,twitter_sentiment,10,Man Gearbox really needs to fix these disappoi...,Negative,2407.0,NaN,Positive
46,twitter_sentiment,13,Check out this big epic streamer!.,Neutral,2408.0,NaN,Positive
63,twitter_sentiment,16,.. 45,Neutral,2411.0,NaN,Negative
...,...,...,...,...,...,...,...
3770,twitter_apple,623499300,Our new @Technogym treadmills getting assemble...,Neutral,NaN,NaN,Negative
3775,twitter_apple,623499305,RT @hoetbh: WHY CAN'T I JUST DATE ME WHY CAN'T...,Neutral,NaN,NaN,Negative
3778,twitter_apple,623499308,#AAPL:Apple products may have Bah! Humbug! hol...,Negative,NaN,NaN,Neutral
3790,twitter_apple,623499320,Five Apple predictions for 2015. I disagree ab...,Positive,NaN,NaN,Neutral


In [32]:
irrelevant = test_ds[test_ds['sentiment'] == 'Irrelevant']
irrelevant

,dataset,tweet_id,text,sentiment,entity,airline,BERT_sentiment
105,twitter_sentiment,27,Appreciate the (sonic) concepts / praxis Valen...,Irrelevant,2418.0,NaN,Irrelevant
106,twitter_sentiment,28,Appreciate by the ( sonic ) electronic concept...,Irrelevant,2418.0,NaN,Neutral
126,twitter_sentiment,35,Loving these new @GhostLifestyle cans!! Anyone...,Irrelevant,2422.0,NaN,Irrelevant
139,twitter_sentiment,37,How the hell are we already into Halloween mon...,Irrelevant,2424.0,NaN,Negative
140,twitter_sentiment,38,How the hell are we already in Halloween month?!.,Irrelevant,2424.0,NaN,Negative
...,...,...,...,...,...,...,...
1832,twitter_apple,623497356,#AAPL:BuzzFeed Is Going To Spend At Least $245...,Irrelevant,NaN,NaN,Neutral
2300,twitter_apple,623497830,"Final #AAPL #PutCallRatios for Friday, Decembe...",Irrelevant,NaN,NaN,Neutral
2457,twitter_apple,623497987,Tax Reform: What to Expect From the New Congre...,Irrelevant,NaN,NaN,Neutral
2572,twitter_apple,623498102,"#AAPL:After Decades Of Consolidation, Wall Str...",Irrelevant,NaN,NaN,Neutral


# Save Model

In [33]:
import pickle
model_fname = 'models/BERT_twitter_model_combined.sav'
pickle.dump(bert_experiment_full['model'], open(model_fname, 'wb'))